In [1]:
# MeCabのインストール
# !apt install mecab libmecab-dev mecab-ipadic-utf8
# !pip install mecab-python3

In [2]:
# !echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

In [3]:
import pandas as pd

import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
#import sklearn
from sklearn.decomposition import PCA 

dataset = pd.read_csv('DataTexat.csv', encoding="utf_8_sig")

FileNotFoundError: [Errno 2] No such file or directory: 'DataTexat.csv'

In [ ]:
pass
dataset['Thanks_strangth'] = dataset['ID']
dataset = dataset.rename(columns = {'title' : 'Text'})

In [ ]:
dataset.head()

In [ ]:
dataset.isnull().sum()

In [ ]:
# type_col = dataset.columns[1:]
type_col =  ['O_Engagement', 'P_WellBeing', 'P_Safety', 'TeamConnect', 'ALL']

df = dataset[type_col]

In [ ]:
def make_SSE(dataset):
    # SSE make
    SSE=[]
    for i in range(1,11):
        model = KMeans(n_clusters=i,
                   init='k-means++',
                   n_init=5,
                   max_iter=10,
                   random_state=123)
        model.fit(df)
        SSE.append(model.inertia_)

    # SSE view
    plt.plot(range(1,11), SSE, marker='o')
    plt.xticks(np.arange(1,11,1))
    plt.xlabel('Number of clusters')
    plt.ylabel('SSE')
    plt.show()

In [ ]:
make_SSE(df)

In [ ]:
def make_class(df):
    
    kmeans_model = KMeans(n_clusters=4, random_state=10).fit(df.iloc[:, 1:])
    labels = kmeans_model.labels_

    color_codes = {0:'#00FF00', 1:'#FF0000', 2:'#0000FF', 3:'#000000'}
    colors = [color_codes[x] for x in labels]

    
    plt.figure(figsize=(8, 8))
    plt.scatter(df['ALL'], df['TeamConnect'], alpha=0.8, color=colors)
#     plt.scatter(df['label3'], df['label2'], alpha=0.8, color=colors)
    plt.title("Principal Component Analysis")
    plt.xlabel("The first principal component score")
    plt.ylabel("The second principal component score")
    plt.show()
    
    return labels

In [ ]:
labels = make_class(df)

In [ ]:
def make_pca(df, labels):
    #pca 
    pca = PCA()
    pca.fit(df.iloc[:, 1:])
    PCA(copy=True, n_components=None, whiten=False)
    
    color_codes = {0:'#00FF00', 1:'#FF0000', 2:'#0000FF', 3:'#000000'}
    colors = [color_codes[x] for x in labels]

    feature = pca.transform(df.iloc[:, 1:])


    plt.figure(figsize=(8, 8))
#     for x, y, name in zip(feature[:, 0], feature[:, 1], df.iloc[:, 0]):
#         plt.text(x, y, name, alpha=0.8, size=10)
    plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8, color=colors, label=False)
    plt.title("Principal Component Analysis")
    plt.xlabel("The first principal component score")
    plt.ylabel("The second principal component score")
    plt.show()

In [ ]:
make_pca(df, labels)

In [ ]:
strangth5 = dataset.loc[dataset['Thanks_strangth']==5]
strangth4 = dataset.loc[dataset['Thanks_strangth']==4]
strangth3 = dataset.loc[dataset['Thanks_strangth']==3]
strangth2 = dataset.loc[dataset['Thanks_strangth']==2]
strangth1 = dataset.loc[dataset['Thanks_strangth']==1]
strangth_list = [strangth1, strangth2, strangth3, strangth4, strangth5]
[len(i) for i in strangth_list]

In [ ]:
strangth1.head()

In [ ]:
dataset['labels'] = labels
dataset.head()

In [ ]:
class3 = dataset.loc[dataset['labels']==3]
class2 = dataset.loc[dataset['labels']==2]
class1 = dataset.loc[dataset['labels']==1]
class0 = dataset.loc[dataset['labels']==0]
class_data = [class0, class1, class2, class3]
[len(i) for i in class_data]

In [ ]:
aa

In [ ]:
import MeCab
# import subprocess
import re
# import torch

# tagger = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
tagger = MeCab.Tagger("-Owakati")
# mecabTagger = MeCab.Tagger("-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

In [ ]:
def make_wakati(sentence):
    #print(sentence)
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', "", sentence)
    #print(sentence)
#     sentence = re.sub(r'[\．_－―─＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    #print(sentence)
    sentence = sentence.replace(' ', '')

    sentence = tagger.parse(sentence)
#     print(sentence)
    
#     tagger.parse("")
    node = tagger.parseToNode(sentence)
    keywords = []
    while node:
        if node.feature.split(",")[0] == u"名詞":
            keywords.append(node.surface)
        elif node.feature.split(",")[0] == u"形容詞":
            keywords.append(node.feature.split(",")[6])
        elif node.feature.split(",")[0] == u"動詞":
            keywords.append(node.feature.split(",")[6])
        node = node.next
#     print(keywords)
    return keywords

In [ ]:
def make_dic(dataset):
    word2index = {}
    word_count = {}
    for title in dataset["Text"]:
        wakati = make_wakati(title)
        for word in wakati:
            if not word in word_count:
                word_count[word] = 0
            word_count[word] += 1
            if word in word2index: continue
            word2index[word] = len(word2index)
    print("vocab size : ", len(word2index))
    word_counts = sorted(word_count.items(), key=lambda x:x[1], reverse=True)
    return word_count

In [ ]:
word_count = make_dic(strangth1)
word_count

---

In [ ]:
# !pip install gensim
#https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

In [ ]:
import gensim
# model = gensim.models.Word2Vec.load('ja/ja.bin') #Word2Vec
# model = gensim.models.KeyedVectors.load_word2vec_format('ja/ja.bin', binary=True) #Word2Vec
model = gensim.models.KeyedVectors.load_word2vec_format('model.vec', binary=False) #FastText

In [ ]:
model.most_similar("感謝", topn=100)

In [ ]:
# model['感謝']
model.most_similar(positive=['日本人'])

In [ ]:
sample = ["賞賛", "応援", "労い", "尊敬", "挨拶", "同意", "自己便益", "願い", "依頼", "指示", "共感", "鼓舞", "喜び", "祈り", "詫び", "雑談"]
data = []
for type_ in sample:
    try:
        tmp = model.most_similar(positive=type_)
        data.append(tmp)
    except:
        print('word’' + type_ + '’not in vocabulary')
        A = type_
sample.remove(A)
print(sample)

In [ ]:
# data

In [ ]:
data = pd.DataFrame(data, index=sample)
data = data.T

In [ ]:
data

In [ ]:
posi_list = ['優れる', '良い','喜ぶ','褒める', 'めでたい','賢い','善い', '適す','天晴',
 '祝う', '功績','賞','嬉しい','喜び','才知','徳', '才能','素晴らしい','芳しい','称える',
 '適切','崇める','助ける','抜きんでる','清水','雄雄しい','仕合せ','幸い','吉兆','秀でる']

nega_list = ['悪い', '死ぬ', '病気', '酷い', '罵る', '浸ける', '卑しい',
 '下手', '苦しむ', '苦しい', '付く', '厳しい', '難しい', '殺す', '難い', '荒荒しい',
 '惨い', '責める', '敵', '背く', '嘲る', '苦しめる', '辛い', '物寂しい', '罰', '不貞腐る',
 '寒い', '下らない', '残念']

In [ ]:
def posi_nega_score(x):
    #ポジティブ度合いの判定
    posi = []
    for i in posi_list:
        try:
            n = model.similarity(i, x)
            posi.append(n)
        except:
            continue
    try:
        posi_mean = sum(posi)/len(posi)
    except:
        posi_mean = 0

    #ネガティブ度合いの判定
    nega = []
    for i in nega_list:
        try:
            n = model.similarity(i, x)
            nega.append(n)
        except:
            continue
    try:
        nega_mean = sum(nega)/len(nega)
    except:
        nega_mean = 0
    if posi_mean > nega_mean:
        return posi_mean
    if nega_mean > posi_mean:
        return -nega_mean
    else:
        return 0

In [ ]:
ddf = pd.read_csv('dataset.csv')
ddf

In [ ]:
#各単語にスコアを割り振る
word['スコア'] = word[0].apply(lambda x : posi_nega_score(x))

import numpy as np
#与えられたスコアを-1から1の範囲に調整
score = np.array(word['スコア'])
score_std = (score - score.min())/(score.max() - score.min())
score_scaled = score_std * (1 - (-1)) + (-1)
word['スコア'] = score_scaled

In [ ]:
nega = word.sort_values('スコア').head(50)

In [ ]:
posi = word.sort_values('スコア', ascending=False).head(50)

In [ ]:
posi

In [ ]:
nega

---